In [ ]:
!pip install tensorflow==2.15

In [ ]:
!pip install opencv-python docopt numpy

  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=c152f4b6ff2889b836b2adccbeeb7ce666cca46dc01e074496a46bebe136cea6
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


# Model Loading and Functions

In [ ]:
import keras
print(keras. __version__)

2.15.0


In [ ]:
import torch
import pickle
place_path = '/content/drive/MyDrive/ML/Models/finalized_model.pkl'
animal_path = '/content/drive/MyDrive/ML/Models/final_animal_model.pkl'


p_model = pickle.load(open(place_path,'rb'))
a_model = pickle.load(open(animal_path,'rb'))

In [ ]:
import cv2
import numpy as np

def load_and_preprocess_place(image_path):
  IMAGE_SIZE = (150, 150)
  # Load the image
  image = cv2.imread(image_path)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = cv2.resize(image, IMAGE_SIZE)

  # Normalize the image
  image = image / 255.0

  return image

def predict_place(p_model, image_path,class_names):
  # Load and preprocess the image
  image = load_and_preprocess_place(image_path)

  # Add batch dimension
  image = np.expand_dims(image, axis=0)

  # Predict the class
  predictions = p_model.predict(image)
  pred_label = np.argmax(predictions, axis=1)[0]

  return class_names[pred_label]

def load_and_preprocess_animal(image_path):
  TARGET_SIZE = (224, 224)
  img = cv2.imread(image_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, TARGET_SIZE)
  img = img.reshape(1,224,224,3)

  return img

def predict_animal(a_model,image_path,labels):
  image = load_and_preprocess_animal(image_path)
  pred = a_model.predict(image)
  pred = np.argmax(pred,axis=1)

  return labels[pred[0]]

In [ ]:
from cryptography.fernet import Fernet

# Generate 16 Fernet keys corresponding to 4-bit numbers (0 to 15)
fernet_keys = {}

# Loop through the 4-bit numbers (0 to 15)
for i in range(16):
    key = Fernet.generate_key()
    fernet_keys[f"{i:04b}"] = key.decode()  # Use 4-bit binary string as the key

# Display the generated keys
for bit_num, key in fernet_keys.items():
    print(f"{bit_num}: {key}")

In [ ]:
from cryptography.fernet import Fernet

sender_key = Fernet.generate_key().decode()
receiver_key = sender_key

print(sender_key)

6718RDKsAU1jjhm_EppvFiNBYczkU_re0ceAh1ppmcc=


In [ ]:
def generate_byte(p_model,a_model,image_path):
  class_names = ['mountain', 'buildings', 'sea', 'forest']
  labels= {0: 'cane', 1: 'gallina', 2: 'pecora', 3: 'ragno'}
  animal = predict_animal(a_model,image_path,labels)
  place = predict_place(p_model,image_path,class_names)
  print(animal,place)
  bit = feature_map_rev["animal"][animal] + feature_map_rev["place"][place]
  return bit

feature_map_rev={
    'place': {'mountain': '00', 'sea': '01', 'buildings': '10', 'forest': '11'},
    'animal': {'cane': '00', 'ragno': '10', 'pecora': '01', 'gallina': '11'}
}

In [ ]:
key_map  = generate_byte(p_model,a_model,"/content/ChickenBuildings1.png")

1/1 [==============================] - 0s 28ms/step
gallina buildings


# Encryption

In [ ]:
from cryptography.fernet import Fernet


fernet = Fernet(sender_key.encode())

# Message to encode
message = "This is a secret message"

# Encode the message
encoded_message = fernet.encrypt(message.encode())  # Encrypted bytes
encoded_message_str = encoded_message.decode()
print(f"Encoded (String format): {encoded_message_str}")

# Decode the message
decoded_message_bytes = encoded_message_str.encode()  # Convert string back to bytes
decoded_message = fernet.decrypt(decoded_message_bytes).decode()

# Decode the message
# decoded_message = fernet.decrypt(encoded_message.encode('utf-8')).decode()
print(f"Decoded: {decoded_message}")


Encoded (String format): gAAAAABnN1vDWG0PcL45LmEMNnfXwJfH339pMpDqp8jzehfmzuJJfiOU55yid9pHRrCkag_OKOuYplm8VLY34iRI9PtFfTkuXU2iFTEAxGLP9Hd6ZgB4H7M=
Decoded: This is a secret message


# Steganography

In [ ]:
"""LSBSteg.py

Usage:
  LSBSteg.py encode -i <input> -o <output> -f <file>
  LSBSteg.py decode -i <input> -o <output>

Options:
  -h, --help                Show this help
  --version                 Show the version
  -f,--file=<file>          File to hide
  -i,--in=<input>           Input image (carrier)
  -o,--out=<output>         Output image (or extracted file)
"""

import cv2
import numpy as np


class SteganographyException(Exception):
    pass


class LSBSteg():
    def __init__(self, im):
        self.image = im
        self.height, self.width, self.nbchannels = im.shape
        self.size = self.width * self.height

        self.maskONEValues = [1,2,4,8,16,32,64,128]
        #Mask used to put one ex:1->00000001, 2->00000010 .. associated with OR bitwise
        self.maskONE = self.maskONEValues.pop(0) #Will be used to do bitwise operations

        self.maskZEROValues = [254,253,251,247,239,223,191,127]
        #Mak used to put zero ex:254->11111110, 253->11111101 .. associated with AND bitwise
        self.maskZERO = self.maskZEROValues.pop(0)

        self.curwidth = 0  # Current width position
        self.curheight = 0 # Current height position
        self.curchan = 0   # Current channel position

    def put_binary_value(self, bits): #Put the bits in the image
        for c in bits:
            val = list(self.image[self.curheight,self.curwidth]) #Get the pixel value as a list
            if int(c) == 1:
                val[self.curchan] = int(val[self.curchan]) | self.maskONE #OR with maskONE
            else:
                val[self.curchan] = int(val[self.curchan]) & self.maskZERO #AND with maskZERO

            self.image[self.curheight,self.curwidth] = tuple(val)
            self.next_slot() #Move "cursor" to the next space

    def next_slot(self):#Move to the next slot were information can be taken or put
        if self.curchan == self.nbchannels-1: #Next Space is the following channel
            self.curchan = 0
            if self.curwidth == self.width-1: #Or the first channel of the next pixel of the same line
                self.curwidth = 0
                if self.curheight == self.height-1:#Or the first channel of the first pixel of the next line
                    self.curheight = 0
                    if self.maskONE == 128: #Mask 1000000, so the last mask
                        raise SteganographyException("No available slot remaining (image filled)")
                    else: #Or instead of using the first bit start using the second and so on..
                        self.maskONE = self.maskONEValues.pop(0)
                        self.maskZERO = self.maskZEROValues.pop(0)
                else:
                    self.curheight +=1
            else:
                self.curwidth +=1
        else:
            self.curchan +=1

    def read_bit(self): #Read a single bit int the image
        val = self.image[self.curheight,self.curwidth][self.curchan]
        val = int(val) & self.maskONE
        self.next_slot()
        if val > 0:
            return "1"
        else:
            return "0"

    def read_byte(self):
        return self.read_bits(8)

    def read_bits(self, nb): #Read the given number of bits
        bits = ""
        for i in range(nb):
            bits += self.read_bit()
        return bits

    def byteValue(self, val):
        return self.binary_value(val, 8)

    def binary_value(self, val, bitsize): #Return the binary value of an int as a byte
        binval = bin(val)[2:]
        if len(binval) > bitsize:
            raise SteganographyException("binary value larger than the expected size")
        while len(binval) < bitsize:
            binval = "0"+binval
        return binval

    def encode_text(self, txt):
        l = len(txt)
        binl = self.binary_value(l, 16) #Length coded on 2 bytes so the text size can be up to 65536 bytes long
        self.put_binary_value(binl) #Put text length coded on 4 bytes
        for char in txt: #And put all the chars
            c = ord(char)
            self.put_binary_value(self.byteValue(c))
        return self.image

    def decode_text(self):
        ls = self.read_bits(16) #Read the text size in bytes
        l = int(ls,2)
        i = 0
        unhideTxt = ""
        while i < l: #Read all bytes of the text
            tmp = self.read_byte() #So one byte
            i += 1
            unhideTxt += chr(int(tmp,2)) #Every chars concatenated to str
        return unhideTxt

    def encode_image(self, imtohide):
        w = imtohide.width
        h = imtohide.height
        if self.width*self.height*self.nbchannels < w*h*imtohide.channels:
            raise SteganographyException("Carrier image not big enough to hold all the datas to steganography")
        binw = self.binary_value(w, 16) #Width coded on to byte so width up to 65536
        binh = self.binary_value(h, 16)
        self.put_binary_value(binw) #Put width
        self.put_binary_value(binh) #Put height
        for h in range(imtohide.height): #Iterate the hole image to put every pixel values
            for w in range(imtohide.width):
                for chan in range(imtohide.channels):
                    val = imtohide[h,w][chan]
                    self.put_binary_value(self.byteValue(int(val)))
        return self.image


    def decode_image(self):
        width = int(self.read_bits(16),2) #Read 16bits and convert it in int
        height = int(self.read_bits(16),2)
        unhideimg = np.zeros((width,height, 3), np.uint8) #Create an image in which we will put all the pixels read
        for h in range(height):
            for w in range(width):
                for chan in range(unhideimg.channels):
                    val = list(unhideimg[h,w])
                    val[chan] = int(self.read_byte(),2) #Read the value
                    unhideimg[h,w] = tuple(val)
        return unhideimg

    def encode_binary(self, data):
        l = len(data)
        if self.width*self.height*self.nbchannels < l+64:
            raise SteganographyException("Carrier image not big enough to hold all the datas to steganography")
        self.put_binary_value(self.binary_value(l, 64))
        for byte in data:
            byte = byte if isinstance(byte, int) else ord(byte) # Compat py2/py3
            self.put_binary_value(self.byteValue(byte))
        return self.image

    def decode_binary(self):
        l = int(self.read_bits(64), 2)
        output = b""
        for i in range(l):
            output += bytearray([int(self.read_byte(),2)])
        return output

# Encryption with steg (First Step)

In [ ]:
key_map  = generate_byte(p_model,a_model,"/content/ChickenBuildings1.png")

1/1 [==============================] - 0s 31ms/step
gallina buildings


In [ ]:
fernet = Fernet(sender_key.encode())  # Initialize Fernet with the selected key

# Message to encode
message = "This is a secret message"

# Encode the message
encoded_message = fernet.encrypt(message.encode())  # Encrypted bytes
encoded_message_str = encoded_message.decode()
print(f"Encoded (String format): {encoded_message_str}")

steg = LSBSteg(cv2.imread("/content/ChickenBuildings1.png"))
img_encoded = steg.encode_text(encoded_message_str)
cv2.imwrite("my_new_image.png", img_encoded)

xor_value_int = int(key_map, 2)
xor_key = (
    chr(ord(sender_key[0]) ^ xor_value_int) + sender_key[1:]
)
sender_key = xor_key
print("XOR-ed Key:", xor_key)

Encoded (String format): gAAAAABnN11rVxEfCMarqv9jloaH7knLANLPPsUClEh0o022VXdtape-NF3w0vmmiGUj9F1ZXhS-tbGPSAFIFaz6oLQw5rsdiBjyLTcKJ2HchHNd3dlK49w=
XOR-ed Key: 8718RDKsAU1jjhm_EppvFiNBYczkU_re0ceAh1ppmcc=


# Decryption with steg

In [ ]:
key_map_out  = generate_byte(p_model,a_model,"/content/my_new_image.png")

1/1 [==============================] - 0s 31ms/step
gallina buildings


In [ ]:

fernet_out = Fernet(receiver_key.encode())
im = cv2.imread("my_new_image.png")

steg = LSBSteg(im)
decoded_message_bytes = steg.decode_text().encode()  # Convert string back to bytes
decoded_message = fernet_out.decrypt(decoded_message_bytes).decode()
print(f"Decoded: {decoded_message}")
# print("Encoded value:",steg.decode_text())

xor_value_int = int(key_map, 2)
xor_key = (
    chr(ord(receiver_key[0]) ^ xor_value_int) + receiver_key[1:]
)
receiver_key = xor_key
print("XOR-ed Key:", xor_key)

Decoded: This is a secret message
XOR-ed Key: 8718RDKsAU1jjhm_EppvFiNBYczkU_re0ceAh1ppmcc=


This shows that both sender and receiver has update the key

In [ ]:
print(sender_key == receiver_key)

True


# Encryption with steg(Step2)

In [ ]:
key_map  = generate_byte(p_model,a_model,"/content/ChickenSea19.png")

1/1 [==============================] - 0s 29ms/step
gallina sea


In [ ]:
fernet = Fernet(sender_key.encode())  # Initialize Fernet with the selected key

# Message to encode
message = "This is the second secret message"

# Encode the message
encoded_message = fernet.encrypt(message.encode())  # Encrypted bytes
encoded_message_str = encoded_message.decode()
print(f"Encoded (String format): {encoded_message_str}")

steg = LSBSteg(cv2.imread("/content/ChickenSea19.png"))
img_encoded = steg.encode_text(encoded_message_str)
cv2.imwrite("my_new_image2.png", img_encoded)

xor_value_int = int(key_map, 2)
xor_key = (
    chr(ord(sender_key[0]) ^ xor_value_int) + sender_key[1:]
)
sender_key = xor_key
print("XOR-ed Key:", xor_key)

Encoded (String format): gAAAAABnN148fChlWAEYmaMQO4YJ26ej3oNYhjWMyd5JpVGZCMgqvFCPG9C9UPhU7WF91S_Nj2Ma4jMxKJXKVmhPpB7-24YukZ9y0-iJWvvOq8rRv3Lafw3_yqLCqV5lcGp_WNP5fEkq
XOR-ed Key: 5718RDKsAU1jjhm_EppvFiNBYczkU_re0ceAh1ppmcc=


# Decrpytion with steg (Step2)

In [ ]:
key_map_out  = generate_byte(p_model,a_model,"/content/my_new_image2.png")

1/1 [==============================] - 0s 28ms/step
gallina sea


In [ ]:

fernet_out = Fernet(receiver_key.encode())
im = cv2.imread("my_new_image2.png")

steg = LSBSteg(im)
decoded_message_bytes = steg.decode_text().encode()  # Convert string back to bytes
decoded_message = fernet_out.decrypt(decoded_message_bytes).decode()
print(f"Decoded: {decoded_message}")
# print("Encoded value:",steg.decode_text())

xor_value_int = int(key_map, 2)
xor_key = (
    chr(ord(receiver_key[0]) ^ xor_value_int) + receiver_key[1:]
)
receiver_key = xor_key
print("XOR-ed Key:", xor_key)

Decoded: This is the second secret message
XOR-ed Key: 5718RDKsAU1jjhm_EppvFiNBYczkU_re0ceAh1ppmcc=


In [ ]:
print(sender_key == receiver_key)

True
